### We will keep changes isolated from the original python code 

### Our original python functions 
we copy and past from TaneAlgorithm.ipynb the funnctions

In [3]:
################################
#We Implort the needed packages
################################
from __future__ import print_function
import numpy as np
import csv
from datetime import datetime
from collections import defaultdict

################################################################
#we copy and past our functions and packages 
################################################################
#Initializing our partitions
partitions = {}
def get_frozenset(x):
    return frozenset([x,])

#We define the function mergee and we explain it's usage in the next cell of code 

def mergee(ps1, ps2):
    merged = []
    m_Row = {}
    for i, p1 in enumerate(ps1):
        for row_j in p1:
#we denote by iRow the jeme row 
            m_Row[row_j] = i
    for p2 in ps2:
        tmp = defaultdict(list)
        for row_j in p2:
            tmp[m_Row[row_j]].append(row_j)
        merged += tmp.values()
    return merged

#Therefore, first check whether the value has been calculated at the head of the function.
#If it has been calculated, the stored value is returned directly, otherwise the classification discussion:
def get_partition(attr):
    if attr in partitions:
        return partitions[attr]
    
#If the length of the attribute set is 0(which means that no attr are used for division)
#so we return an empty set.

    if len(attr) == 0:
        partitions[attr] = []
        #If the length of the attribute set is 1 (the attribute will be used as the basis for division)
#and a map is used to aggregate the equivalence classes under the division basis. 
#The set of these equivalence classes is the desired result.

    elif len(attr) == 1:
        iAttr = tuple(attr)[0]
        d = defaultdict(list)
        for index, row in enumerate(table):
            d[row[iAttr]].append(index)
        partitions[attr] = d.values()
#The length of the attribute set is greater than or equal to 2 (the length is set to l), 
#and the current division result can be obtained by using the previous division result. 
#The specific implementation method is:
#Obtain the two divisions corresponding to any two different attribute subsets of 
#length l-1 in the current attribute set.
#The product of these two divisions is the desired result. 
#For the detailed process, see the function mergee.

    else:
        attr_tuple = tuple(attr)
        ps1 = get_partition(frozenset(attr_tuple[0:-1]))
        ps2 = get_partition(frozenset(attr_tuple[0:-2] + attr_tuple[-1:]))
        partitions[attr] = mergee(ps1, ps2)

    return partitions[attr]


#test if X\{E} -> E is valid
#X is a set of number, E is a number
def validity_test(X, E):
    return len(get_partition(X - {E})) == len(get_partition(X))


def dependencies_discovery(L): # L is a set of tuple of number
    L_new = L.copy()
    for X in L:
        Xs = frozenset(X)
        RHS[X] = R
        for E in Xs:
            RHS[X] = RHS[X] & RHS[Xs - {E}]
        for E in RHS[X] & Xs:
            if validity_test(Xs, E):
                fds.append((Xs - {E}, E))
                RHS[X] -= get_frozenset(E)
                RHS[X] = RHS[X] & Xs
        if len(RHS[X]) == 0:
            L_new.remove(X)

    return L_new


def generate_next_level(L):#----> next level or (layer)
    # list comprehension?
    Ln = set([])
    for l1 in L:
        for l2 in L:
            if l1 != l2 and len(l1 - l2) == 1:
                Ln.add(l1 | l2)
    return Ln

 #create an output file 

def output_fd(fds):
    with open(output_filename, 'w') as f:
        #creating an output file of the desired format in write mode
                                         
        fds_sorted = sorted(fds)
        for fd in fds_sorted:
            str = ''
            for l in sorted(list(fd[0])):
                str += '%d ' % (l+1)
            str += '-> %d\n' % (fd[1]+1)
            f.write(str)
            #also with creating the output file & we print the result here in the jupyter notebook
            print(str, end='')

In [4]:
#we begin by creating a SparkContext that loads settings from system properties
from pyspark import SparkContext, SparkConf

In [44]:
################################
#Our main 
################################

#The entry point into all functionality in Spark is the SparkSession class. 
#To create a basic SparkSession, just use SparkSession.builder:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local") \
    .appName("Amen_TANE_algorithm") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()



input_filename = 'iris.csv'
output_filename = 'output-iris-Spark.txt'


#Apply the same context of the python implementation BEFORE
sc = spark.sparkContext
# Load a text file and convert each line to a Row.
rdd = sc.textFile(input_filename)
# Each line is converted to a tuple.
rdd = rdd.mapPartitions(lambda x: csv.reader(x))
table=rdd.collect()
print(table)

[['5.1', '3.5', '1.4', '0.2', 'Iris-setosa'], ['4.9', '3.0', '1.4', '0.2', 'Iris-setosa'], ['4.7', '3.2', '1.3', '0.2', 'Iris-setosa'], ['4.6', '3.1', '1.5', '0.2', 'Iris-setosa'], ['5.0', '3.6', '1.4', '0.2', 'Iris-setosa'], ['5.4', '3.9', '1.7', '0.4', 'Iris-setosa'], ['4.6', '3.4', '1.4', '0.3', 'Iris-setosa'], ['5.0', '3.4', '1.5', '0.2', 'Iris-setosa'], ['4.4', '2.9', '1.4', '0.2', 'Iris-setosa'], ['4.9', '3.1', '1.5', '0.1', 'Iris-setosa'], ['5.4', '3.7', '1.5', '0.2', 'Iris-setosa'], ['4.8', '3.4', '1.6', '0.2', 'Iris-setosa'], ['4.8', '3.0', '1.4', '0.1', 'Iris-setosa'], ['4.3', '3.0', '1.1', '0.1', 'Iris-setosa'], ['5.8', '4.0', '1.2', '0.2', 'Iris-setosa'], ['5.7', '4.4', '1.5', '0.4', 'Iris-setosa'], ['5.4', '3.9', '1.3', '0.4', 'Iris-setosa'], ['5.1', '3.5', '1.4', '0.3', 'Iris-setosa'], ['5.7', '3.8', '1.7', '0.3', 'Iris-setosa'], ['5.1', '3.8', '1.5', '0.3', 'Iris-setosa'], ['5.4', '3.4', '1.7', '0.2', 'Iris-setosa'], ['5.1', '3.7', '1.5', '0.4', 'Iris-setosa'], ['4.6', '

In [45]:
maxL = len(table[0])
R = frozenset(range(maxL))
RHS = {frozenset([]): R}
fds = []


In [46]:
if __name__ == '__main__' :
    
    L = frozenset(map(get_frozenset, R))
    L = dependencies_discovery(L)
    for i in range(1, maxL):
        L = dependencies_discovery(generate_next_level(L))
        print('dependencies discovery at level: %d' %(i+1))

   
    output_fd(fds)
   
    #print in the notebook
    print('The output-adult-Spark.txt has been created')

dependencies discovery at level: 2
dependencies discovery at level: 3
dependencies discovery at level: 4
dependencies discovery at level: 5
1 3 4 -> 5
2 3 4 -> 5
1 2 4 -> 5
1 2 3 -> 5
The output-adult-Spark.txt has been created
